In [1]:
#importing pyspark
import pyspark

In [2]:
#importing sparksession
from pyspark.sql import SparkSession

In [3]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("business_case_sql").getOrCreate()

In [9]:
#To create dataframe form External datasets
AirlineDF = spark.read.option("header", "true").csv("airlines1.csv")

To register dataframe as a table

In [10]:
AirlineDF.registerTempTable("AirlineTable")

/usr/local/spark/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [11]:
#how to read the register table
spark.sql("select * from AirlineTable").show(5)

+---+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+--------------------+---------+-------------+--------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----

In [12]:
#count number of rows
spark.sql("select count(*) from AirlineTable").show()

+--------+
|count(1)|
+--------+
|   50001|
+--------+



In [13]:
#how to see all columns with datatype
spark.sql("describe AirlineTable").show(50)

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                 _c0|   string|   null|
|                Year|   string|   null|
|             Quarter|   string|   null|
|               Month|   string|   null|
|          DayofMonth|   string|   null|
|           DayOfWeek|   string|   null|
|          FlightDate|   string|   null|
|   Reporting_Airline|   string|   null|
|DOT_ID_Reporting_...|   string|   null|
|IATA_CODE_Reporti...|   string|   null|
|         Tail_Number|   string|   null|
|Flight_Number_Rep...|   string|   null|
|     OriginAirportID|   string|   null|
|  OriginAirportSeqID|   string|   null|
|  OriginCityMarketID|   string|   null|
|              Origin|   string|   null|
|      OriginCityName|   string|   null|
|         OriginState|   string|   null|
|     OriginStateFips|   string|   null|
|     OriginStateName|   string|   null|
|           OriginWac|   string|   null|
|       DestAirp

In [14]:
# select some columns form table
spark.sql("select origin,destcityname from AirlineTable").show(5)

+------+--------------------+
|origin|        destcityname|
+------+--------------------+
|   MSP|  Salt Lake City, UT|
|   MKE|         Orlando, FL|
|   GJT|Dallas/Fort Worth...|
|   LAX|         Detroit, MI|
|   EWR|       Charlotte, NC|
+------+--------------------+
only showing top 5 rows



In [15]:
# average departure delay 
spark.sql("select avg(DepDelayMinutes) from AirlineTable").show()

+--------------------+
|avg(DepDelayMinutes)|
+--------------------+
|  10.851455322613475|
+--------------------+



In [16]:
#calculate overall flights delay 
spark.sql("select Flight_Number_Reporting_Airline,DepDelay+ArrDelay  from AirlineTable").show()

+-------------------------------+---------------------+
|Flight_Number_Reporting_Airline|(DepDelay + ArrDelay)|
+-------------------------------+---------------------+
|                            675|                 42.0|
|                            671|                 -2.0|
|                           3297|                 11.0|
|                           1806|                -20.0|
|                            465|                 83.0|
|                           1198|                 11.0|
|                           1431|                 -2.0|
|                           3459|                435.0|
|                           7233|                 12.0|
|                           5932|                 45.0|
|                           2135|                  8.0|
|                           1463|                 -5.0|
|                            830|                -13.0|
|                           6474|                -23.0|
|                            674|               

In [17]:
# count total number of flights by origin airport and total avg and maximum DepDelay time of each airport
spark.sql("select origin,count(*),avg(DepDelayMinutes),max(DepDelayMinutes) from AirlineTable where group by origin").show()

+------+--------+--------------------+--------------------+
|origin|count(1)|avg(DepDelayMinutes)|max(DepDelayMinutes)|
+------+--------+--------------------+--------------------+
|   ABE|      49|   6.659574468085107|                44.0|
|   ABI|      11|  0.5555555555555556|                 4.0|
|   ABQ|     290|   9.041958041958042|                 9.0|
|   ABR|       3|               284.0|               852.0|
|   ABY|       3|                20.0|                30.0|
|   ACK|       1|                 0.0|                 0.0|
|   ACT|      16|              3.1875|                33.0|
|   ACV|      14|   7.071428571428571|                 7.0|
|   ACY|       5|                 7.6|                38.0|
|   ADK|       2|                 6.5|                13.0|
|   ADQ|       4|   4.666666666666667|                14.0|
|   AEX|      11|   8.090909090909092|                34.0|
|   AGS|      27|  23.814814814814813|                64.0|
|   AKN|       2|                 9.5|  

In [18]:
# calculate origin state wise cancelled flights
spark.sql("select OriginState,count(Cancelled) from AirlineTable where Cancelled = 1  group by originstate").show(5)

+-----------+----------------+
|OriginState|count(Cancelled)|
+-----------+----------------+
|         SC|               8|
|         AZ|              12|
|         LA|               5|
|         MN|              26|
|         NJ|              27|
+-----------+----------------+
only showing top 5 rows



In [19]:
#pass condition on two columns
spark.sql("select origin,count(*) from AirlineTable where year = 2010 and month = 6  group by origin order by count(*) desc").show()

+------+--------+
|origin|count(1)|
+------+--------+
|   ATL|       8|
|   ORD|       7|
|   PHX|       6|
|   DFW|       6|
|   MCO|       6|
|   LAX|       5|
|   IAH|       5|
|   CLE|       4|
|   CLT|       3|
|   BOS|       3|
|   MIA|       3|
|   SFO|       3|
|   CMH|       3|
|   LAS|       3|
|   MEM|       3|
|   TPA|       3|
|   IAD|       3|
|   EWR|       2|
|   DEN|       2|
|   SMF|       2|
+------+--------+
only showing top 20 rows



In [20]:
# pass more than two condition on difftent columns
spark.sql("select DOT_ID_Reporting_Airline from AirlineTable where year = 2015 and month = 12 and origin = 'JFK'").show()

+------------------------+
|DOT_ID_Reporting_Airline|
+------------------------+
|                   20409|
|                   19805|
+------------------------+



In [21]:
#rank and number of diverted flight by origin 
spark.sql("select origin,sum(diverted),RANK() over(order by sum(diverted) desc) AS RANKING from AirlineTable group by origin").show()

+------+-------------+-------+
|origin|sum(diverted)|RANKING|
+------+-------------+-------+
|   DEN|          6.0|      1|
|   ATL|          6.0|      1|
|   ORD|          6.0|      1|
|   MSP|          6.0|      1|
|   DFW|          5.0|      5|
|   FLL|          5.0|      5|
|   CVG|          4.0|      7|
|   CLT|          4.0|      7|
|   LAX|          4.0|      7|
|   MCO|          4.0|      7|
|   CMH|          3.0|     11|
|   SEA|          3.0|     11|
|   PDX|          3.0|     11|
|   BWI|          3.0|     11|
|   STL|          3.0|     11|
|   PIT|          3.0|     11|
|   DCA|          2.0|     17|
|   PSG|          2.0|     17|
|   BNA|          2.0|     17|
|   IAH|          2.0|     17|
+------+-------------+-------+
only showing top 20 rows



# Case Study 1
To calulate year wise on time flight performance 

In [22]:
df1 = spark.sql("select year,count(*) AS TOTAL_FLIGHT ,count(arrdelay==0) AS TOTAL_ON_TIME,sum(cancelled) AS TOTAL_CANCELLED,sum(diverted) AS TOTAL_DIRVERTED from AirlineTable where group by year;")

In [23]:
df1.registerTempTable("AirlineTable1")

In [24]:
spark.sql("select * from AirlineTable1").show()

+----+------------+-------------+---------------+---------------+
|year|TOTAL_FLIGHT|TOTAL_ON_TIME|TOTAL_CANCELLED|TOTAL_DIRVERTED|
+----+------------+-------------+---------------+---------------+
|1987|         339|          336|            3.0|            0.0|
|2016|        1502|         1484|           17.0|            1.0|
|2020|         462|          438|           24.0|            0.0|
|2012|        1519|         1497|           20.0|            2.0|
|1988|        1310|         1293|           15.0|            2.0|
|2019|        1900|         1862|           34.0|            4.0|
|2017|        1449|         1421|           22.0|            6.0|
|2014|        1513|         1476|           31.0|            6.0|
|2013|        1690|         1663|           22.0|            5.0|
|2005|        1812|         1776|           33.0|            3.0|
|2000|        1477|         1432|           42.0|            3.0|
|2002|        1371|         1360|           11.0|            0.0|
|2009|    

In [25]:
# performance in percentage
spark.sql("select year,TOTAL_FLIGHT,TOTAL_ON_TIME,TOTAL_ON_TIME*100/TOTAL_FLIGHT AS ON_TIME_PER,TOTAL_CANCELLED,TOTAL_CANCELLED*100/TOTAL_FLIGHT AS CANCELLED_PER,TOTAL_DIRVERTED,TOTAL_DIRVERTED*100/TOTAL_FLIGHT AS DIVERTED_PER from AirlineTable1;").show()

+----+------------+-------------+-----------------+---------------+------------------+---------------+-------------------+
|year|TOTAL_FLIGHT|TOTAL_ON_TIME|      ON_TIME_PER|TOTAL_CANCELLED|     CANCELLED_PER|TOTAL_DIRVERTED|       DIVERTED_PER|
+----+------------+-------------+-----------------+---------------+------------------+---------------+-------------------+
|1987|         339|          336|99.11504424778761|            3.0|0.8849557522123894|            0.0|                0.0|
|2016|        1502|         1484|98.80159786950732|           17.0|1.1318242343541944|            1.0|0.06657789613848203|
|2020|         462|          438| 94.8051948051948|           24.0| 5.194805194805195|            0.0|                0.0|
|2012|        1519|         1497|98.55167873601053|           20.0| 1.316655694535879|            2.0| 0.1316655694535879|
|1988|        1310|         1293|98.70229007633588|           15.0|1.1450381679389312|            2.0|0.15267175572519084|
|2019|        19

In [26]:
#To calulate origincityname wise on time flight performance 

In [27]:
df2 = spark.sql("select origincityname,count(*) AS TOTAL_FLIGHT ,count(arrdelay==0) AS TOTAL_ON_TIME,sum(cancelled) AS TOTAL_CANCELLED,sum(diverted) AS TOTAL_DIRVERTED from AirlineTable where group by origincityname;")

In [28]:
df2.registerTempTable("AirlineTable2")

In [29]:
spark.sql("select * from AirlineTable2").show()

+------------------+------------+-------------+---------------+---------------+
|    origincityname|TOTAL_FLIGHT|TOTAL_ON_TIME|TOTAL_CANCELLED|TOTAL_DIRVERTED|
+------------------+------------+-------------+---------------+---------------+
|   Gainesville, FL|          18|           18|            0.0|            0.0|
|      Richmond, VA|         146|          141|            5.0|            0.0|
|        Tucson, AZ|         157|          155|            2.0|            0.0|
|       Ontario, CA|         270|          266|            3.0|            1.0|
|        Pierre, SD|           1|            1|            0.0|            0.0|
|       Medford, OR|          21|           21|            0.0|            0.0|
|  Myrtle Beach, SC|          41|           40|            1.0|            0.0|
|  Palm Springs, CA|          85|           80|            3.0|            2.0|
|       Redding, CA|           8|            8|            0.0|            0.0|
|       Durango, CO|           9|       

In [30]:
spark.sql("select origincityname,TOTAL_FLIGHT,TOTAL_ON_TIME,TOTAL_ON_TIME*100/TOTAL_FLIGHT AS ON_TIME_PER,TOTAL_CANCELLED,TOTAL_CANCELLED*100/TOTAL_FLIGHT AS CANCELLED_PER,TOTAL_DIRVERTED,TOTAL_DIRVERTED*100/TOTAL_FLIGHT AS DIVERTED_PER from AirlineTable2;").show()

+------------------+------------+-------------+-----------------+---------------+------------------+---------------+-------------------+
|    origincityname|TOTAL_FLIGHT|TOTAL_ON_TIME|      ON_TIME_PER|TOTAL_CANCELLED|     CANCELLED_PER|TOTAL_DIRVERTED|       DIVERTED_PER|
+------------------+------------+-------------+-----------------+---------------+------------------+---------------+-------------------+
|   Gainesville, FL|          18|           18|            100.0|            0.0|               0.0|            0.0|                0.0|
|      Richmond, VA|         146|          141|96.57534246575342|            5.0|3.4246575342465753|            0.0|                0.0|
|        Tucson, AZ|         157|          155|98.72611464968153|            2.0|1.2738853503184713|            0.0|                0.0|
|       Ontario, CA|         270|          266|98.51851851851852|            3.0|1.1111111111111112|            1.0|0.37037037037037035|
|        Pierre, SD|           1|        

In [31]:
# multiple group by columns
spark.sql("select origin,origincityname,count(*) from AirlineTable group by origin,origincityname;").show()

+------+--------------------+--------+
|origin|      origincityname|count(1)|
+------+--------------------+--------+
|   COS|Colorado Springs, CO|     100|
|   SDF|      Louisville, KY|     175|
|   CLL|College Station/B...|      10|
|   PIR|          Pierre, SD|       1|
|   MSN|         Madison, WI|      81|
|   EVV|      Evansville, IN|      18|
|   CMI|Champaign/Urbana, IL|      10|
|   GTR|        Columbus, MS|       2|
|   RIC|        Richmond, VA|     146|
|   STX|   Christiansted, VI|      11|
|   FLO|        Florence, SC|       4|
|   DCA|      Washington, DC|     704|
|   IPL|       El Centro, CA|       1|
|   CKB|Clarksburg/Fairmo...|       1|
|   GRI|    Grand Island, NE|       3|
|   ITO|            Hilo, HI|      26|
|   MSP|     Minneapolis, MN|    1128|
|   AVL|       Asheville, NC|      29|
|   CHS|      Charleston, SC|     109|
|   PIT|      Pittsburgh, PA|     641|
+------+--------------------+--------+
only showing top 20 rows



In [32]:
# rank of origin by number of flights
spark.sql("select rank() over (ORDER BY count(*)desc),origin,origincityname,count(*) from AirlineTable where group by origin,origincityname;").show()

+------------------------------------------------------------------------------------------------+------+--------------------+--------+
|RANK() OVER (ORDER BY count(1) DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)|origin|      origincityname|count(1)|
+------------------------------------------------------------------------------------------------+------+--------------------+--------+
|                                                                                               1|   ATL|         Atlanta, GA|    2669|
|                                                                                               2|   ORD|         Chicago, IL|    2536|
|                                                                                               3|   DFW|Dallas/Fort Worth...|    2188|
|                                                                                               4|   LAX|     Los Angeles, CA|    1718|
|                                               

In [33]:
#total air time and distance travel by flights
spark.sql("select AirTime,Distance from AirlineTable").show()

+-------+--------+
|AirTime|Distance|
+-------+--------+
|  153.0|   991.0|
|  141.0|  1066.0|
|  103.0|   773.0|
|  220.0|  1979.0|
|   80.0|   529.0|
|   28.0|   190.0|
|   94.0|   563.0|
|   35.0|   192.0|
|   59.0|   316.0|
|  114.0|   793.0|
|   null|   109.0|
|   77.0|   562.0|
|   null|  1045.0|
|   95.0|   677.0|
|   99.0|   733.0|
|   null|   278.0|
|   24.0|    98.0|
|  102.0|   689.0|
|  255.0|  2288.0|
|   null|   373.0|
+-------+--------+
only showing top 20 rows



In [34]:
#speed of flights in miles/hour

In [35]:
df3 = spark.sql("select AirTime/60 As AirTime_Hour,Distance from AirlineTable")

In [36]:
df3.registerTempTable("AirlineTable3")

In [37]:
spark.sql("select AirTime_Hour,Distance, Distance/AirTime_Hour from AirlineTable3").show()

+------------------+--------+-------------------------+
|      AirTime_Hour|Distance|(Distance / AirTime_Hour)|
+------------------+--------+-------------------------+
|              2.55|   991.0|        388.6274509803922|
|              2.35|  1066.0|        453.6170212765957|
|1.7166666666666666|   773.0|       450.29126213592235|
|3.6666666666666665|  1979.0|        539.7272727272727|
|1.3333333333333333|   529.0|                   396.75|
|0.4666666666666667|   190.0|        407.1428571428571|
|1.5666666666666667|   563.0|       359.36170212765956|
|0.5833333333333334|   192.0|        329.1428571428571|
|0.9833333333333333|   316.0|       321.35593220338984|
|               1.9|   793.0|        417.3684210526316|
|              null|   109.0|                     null|
|1.2833333333333334|   562.0|        437.9220779220779|
|              null|  1045.0|                     null|
|1.5833333333333333|   677.0|        427.5789473684211|
|              1.65|   733.0|       444.24242424